In [1]:
from paz.backend.image import load_image, show_image
from paz.applications import HaarCascadeFrontalFace, MiniXceptionFER
import paz.processors as pr
from paz.backend.image.image import crop_image
from paz.processors.detection import CropBoxes2D
import numpy as np
from pathlib import Path
from paz.abstract import SequentialProcessor
from retinaface import RetinaFace
from paz.abstract.messages import Box2D

In [14]:
class process_front_images():
    def __init__(self):
        self.folder_path = Path('DMD\s3_rgb_face').__str__()
        self.result = np.array([[0, 0]]) # 2D array with element [frame_number, emotion]
        self.frame_offset = 10
    def get_emotion(self):
        for frame_number in range(1+self.frame_offset, 10+self.frame_offset): # starts from 0 + frame_offset
            path = self.folder_path + '\%(number)06d.png' % {"number": frame_number}
            image = load_image(path)
    
            print('--------Frame {fnumber}------'.format(fnumber = frame_number))
            crop = CropBoxes2D()
            face = RetinaFace.detect_faces(path)
            coordinates = face['face_1']['facial_area']
            boxes2D = [Box2D(coordinates, score=0)]
            print('Boxes: ', boxes2D, type(boxes2D))
            if (len(boxes2D) == 0):
                self.result = np.append(self.result, [[frame_number, 'nofacedetected']], axis=0)
                continue
                
            elif (len(boxes2D) >= 2):
                areas = np.array([])
                for box in boxes2D:
                    area = (box.coordinates[2]-box.coordinates[0]) * (box.coordinates[3]-box.coordinates[1])
                    areas = np.append(areas, area)
                indices_excluding_max = np.where(areas != np.amax(areas))[0]
                print('Indices to remove', indices_excluding_max)
                for i in indices_excluding_max:
                    boxes2D = np.delete(boxes2D, i)
                print('After removing smaller boxes: ', boxes2D)            
            
            cropped_images = crop(image, boxes2D)
            #show_image(cropped_images[0])
            classify = MiniXceptionFER()
            emotion = classify(cropped_images[0])['class_name']
            print([frame_number, emotion])
            self.result = np.append(self.result, [[frame_number, emotion]], axis=0)
        print('----------------Done getting emotion------------------')
        return self.result
    
front_image_processor = process_front_images()
emotion_result = front_image_processor.get_emotion()
print('Front face emotion labeling: ', emotion_result, emotion_result.shape)

--------Frame 11------
Boxes:  [Box2D(497, 307, 717, 576, 0, None)] <class 'list'>
[11, 'sad']
--------Frame 12------
Boxes:  [Box2D(497, 311, 718, 578, 0, None)] <class 'list'>
[12, 'sad']
--------Frame 13------
Boxes:  [Box2D(498, 309, 717, 580, 0, None)] <class 'list'>
[13, 'sad']
--------Frame 14------
Boxes:  [Box2D(497, 313, 718, 580, 0, None)] <class 'list'>
[14, 'sad']
--------Frame 15------
Boxes:  [Box2D(498, 312, 717, 582, 0, None)] <class 'list'>
[15, 'sad']
--------Frame 16------
Boxes:  [Box2D(497, 313, 716, 584, 0, None)] <class 'list'>
[16, 'sad']
--------Frame 17------
Boxes:  [Box2D(497, 314, 716, 585, 0, None)] <class 'list'>
[17, 'sad']
--------Frame 18------
Boxes:  [Box2D(497, 314, 716, 586, 0, None)] <class 'list'>
[18, 'sad']
--------Frame 19------
Boxes:  [Box2D(498, 314, 715, 590, 0, None)] <class 'list'>
[19, 'neutral']
----------------Done getting emotion------------------
Front face emotion labeling:  [['0' '0']
 ['11' 'sad']
 ['12' 'sad']
 ['13' 'sad']
 ['

In [15]:
class process_side_images():
    def __init__(self):
        self.folder_path = Path('DMD\s3_rgb_body').__str__()
        self.result = np.array([['0', '0']]) # path format?
        self.frame_offset = 10
    def get_face_image(self):
        for frame_number in range(1, 10):
            path = self.folder_path + '\%(number)06d.png' % {"number": frame_number}
            image = load_image(path)
    
            print('--------Frame {fnumber}------'.format(fnumber = frame_number))
            crop = CropBoxes2D()
            face = RetinaFace.detect_faces(path)
            coordinates = face['face_1']['facial_area']
            boxes2D = [Box2D(coordinates, score=0)]
            print('Boxes: ', boxes2D, type(boxes2D))
            if (len(boxes2D) == 0):
                self.result = np.append(self.result, [[frame_number, 'nofacedetected']], axis=0)
                continue
                
            elif (len(boxes2D) >= 2):
                areas = np.array([])
                for box in boxes2D:
                    area = (box.coordinates[2]-box.coordinates[0]) * (box.coordinates[3]-box.coordinates[1])
                    areas = np.append(areas, area)
                indices_excluding_max = np.where(areas != np.amax(areas))[0]
                print('Indices to remove', indices_excluding_max)
                for i in indices_excluding_max:
                    boxes2D = np.delete(boxes2D, i)
                print('After removing smaller boxes: ', boxes2D)            
            
            if (emotion_result[frame_number + self.frame_offset][1] == 'nofacedetected'):
                    self.result = np.append(self.result, [[frame_number, 'nofacedetected']], axis=0)
            
            else:   # both dataset have face detected for this frame number              
                cropped_images = crop(image, boxes2D)
                #show_image(cropped_images[0])
                name = 'img.%(number)06d.png'% {'number': frame_number}
                cropped_images.save(name)
                print(name)
side_processor = process_side_images()
side_processor.get_face_image()

--------Frame 1------
Boxes:  [Box2D(393, 25, 541, 238, 0, None)] <class 'list'>


IndexError: index 11 is out of bounds for axis 0 with size 10

### Code for detecting faces using PAZ

Poor detection performance
    4 out of 5 images were detected wrongly - boxes not appears around the face

In [ ]:
path = self.folder_path + '\%(number)06d.png' % {"number": frame_number}
            image = load_image(path)
    
            print('--------Frame {fnumber}------'.format(fnumber = frame_number))
            detect = HaarCascadeFrontalFace(draw=False)
            crop = CropBoxes2D()
            inference = detect(image)
            boxes2D = inference['boxes2D']
            print('Boxes: ', boxes2D, type(boxes2D))